<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/github/eywalker/LVIV-2021/blob/main/notebooks/DeepLearing%20in%20Neuroscience.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/eywalker/LVIV-2021/blob/main/notebooks/DeepLearing%20in%20Neuroscience.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Welcome to Deep Learning in Neuroscience by Edgar Y. Walker

This is a Jupyter notebook to accompany the course on "Deep Learning in Neuroscience" taught as part of the Lviv Data Science Summer School 2021. This notebook as well as any other relevant information can be found in the [GitHub repository](https://github.com/eywalker/lviv-2021)!

In this course, we will learn how deep learning is getting utilized in studying neuroscience, specifically in building models of neurons to complex sensory inputs such as natural images. We will start by going through some neuroscience primer. We will then get our hands dirty by taking real neuronal responses recorded from mouse primary visual cortex (V1) as the mouse observes a bunch of natural images and developing a model to predict these responses. By the end of this course, you will gain some basic familiarity in utilizing deep learning models to predict responses of 1000s of neurons to natural images!

## Preparing the environment

#### <font color='red'>NOTE: Please run this section at the very beginning of the first session!</font>

Before we get to dive in and learn how deep learning is used in neuroscience and get your first neural predictive model trained, we need to install some prerequisite packages and download some neuronal data!

### Getting the code

We are going to primarily use [PyTorch](https://pytorch.org) to build, train and evaluate our deep learning models and I am going to assume some familiarity with PyTorch already.

Also to be able to handle the dataset containing neuronal activities, we are going to make our life easier by using a few existing libraries. I have prepared a library called [lviv2021](https://github.com/eywalker/lviv2021). This library has a dependency on [neuralpredictors](https://github.com/sinzlab/neuralpredictors), which is a collection of PyTorch layers, tools and other utilities that would prove helpful to train networks to predict neuronal responses.

Let's go ahead and install this inside the Colab environment.

In [ ]:
# Install PyTorch dependency
!pip3 install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html
    
# Install 
!pip3 install git+https://github.com/eywalker/lviv-2021.git

### Getting the dataset

We are going to use the dataset made available for our recent paper [Lurz et al. ICLR 2021](https://github.com/sinzlab/Lurz_2020_code), predicting responses of mouse visual cortex to natural images. 

The dataset can take anywhere from 5-10 min to download, so please be sure to **run the following at the very beginning of the session!** We are going to first spend some time learning the basics of computational neuroscience in the study of system identification. It would be best that you let the download take place while we go over the neursocience primer so that it will be ready when we come back here to get our hands dirty!

To download the data, simply execute the following cell, and let it run till completion.

In [ ]:
!git clone https://gin.g-node.org/cajal/Lurz2020.git /content/data

# Developing models of neural population responses to natural images

Now that you have been primed with just enough background neuroscience, let's get our hand dirty and try to build our first neural predictive models.

As part of the setup, we have downloaded a 2-photon imaging dataset from mouse primary visual cortex as we present 1000s of natural images (if not done yet, please do so immediately by stepping through the beginning sections of this notebook).

In [1]:
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import matplotlib.pyplot as plt

## Navigating the neuroscience data

As with any data science project, you must start by understanding your data! Let's take some time to navigate the data you downloaded.

In [2]:
ls ./data/static20457-5-9-preproc0/

change.log  config.json  data/  meta/


In [3]:
ls ./data/static20457-5-9-preproc0/data

behavior/  images/  pupil_center/  responses/


In [4]:
ls ./data/static20457-5-9-preproc0/data/responses | head -30

0.npy
1.npy
10.npy
100.npy
1000.npy
1001.npy
1002.npy
1003.npy
1004.npy
1005.npy
1006.npy
1007.npy
1008.npy
1009.npy
101.npy
1010.npy
1011.npy
1012.npy
1013.npy
1014.npy
1015.npy
1016.npy
1017.npy
1018.npy
1019.npy
102.npy
1020.npy
1021.npy
1022.npy
1023.npy
ls: write error: Broken pipe


In [5]:
ls ./data/static20457-5-9-preproc0/data/images | head -30

0.npy
1.npy
10.npy
100.npy
1000.npy
1001.npy
1002.npy
1003.npy
1004.npy
1005.npy
1006.npy
1007.npy
1008.npy
1009.npy
101.npy
1010.npy
1011.npy
1012.npy
1013.npy
1014.npy
1015.npy
1016.npy
1017.npy
1018.npy
1019.npy
102.npy
1020.npy
1021.npy
1022.npy
1023.npy
ls: write error: Broken pipe


You can see that both responses and contained in collections of `numpy` files named like `1.npy` or `31.npy`. The number here corresponds to a specific **trial** or simply different image presentation during an experiment.

Let's take a look at some of these files.

### Loading data files one at a time

Let's pick some trial and load the image as well as the response

In [6]:
trial_idx = 1100
trial_image = np.load(f'./data/static20457-5-9-preproc0/data/images/{trial_idx}.npy')
trial_responses = np.load(f'./data/static20457-5-9-preproc0/data/responses/{trial_idx}.npy')

The image is shaped as $\text{channel} \times \text{height} \times \text{width}$

In [7]:
trial_image.shape

(1, 36, 64)

In [8]:
plt.imshow(trial_image.squeeze(), cmap='gray', vmin=0, vmax=255)
plt.axis('off')

(-0.5, 63.5, 35.5, -0.5)

In contrast, the shape of `trial_response` is simply the number of neurons

In [9]:
trial_responses.shape

(5335,)

In [10]:
trial_responses.min() # responses are practically always >= 0

-5.409307320414307e-10

In [11]:
trial_responses.max()

53.39189366955226

In [12]:
fig, ax = plt.subplots(1, 1, dpi=150)
ax.hist(trial_responses, 100);

You can see most neuron's responses stay very close to 0 - signifying no activity.

### Loading the entire dataset

While we can inspect the image and the corresponding neural population responses one image at a time, this is quite cumbersome and also impractical for use in network training. Fortunately, the `lviv` package provides us with a convenience function that will help to load the entire dataset as PyTorch dataloaders.

In [13]:
from lviv.dataset import load_dataset

As we prepare the dataloaders, we get to specify the batch size.

In [14]:
dataloaders = load_dataset(path = '/content/data/static20457-5-9-preproc0', batch_size=60)

The function returns a dictionary consisting of three dataloaders for training, validation, and test set.

In [15]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7fb9d6889eb0>,
 'validation': <torch.utils.data.dataloader.DataLoader at 0x7fb9d6889c10>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7fb9d6889ee0>}

Let's specifically look at the trainset dataloader

In [16]:
train_loader = dataloaders['train']

Total number of images can be checked as follows:

In [17]:
len(train_loader.sampler)

4472

We can inspect what it returns per batch:

In [18]:
images, responses = next(iter(train_loader))

In [19]:
images.shape

torch.Size([60, 1, 36, 64])

In [20]:
responses.shape

torch.Size([60, 5335])

As expected, you can see it returns a batch size of 60 images and responses for all neurons.

Similar inspection can be done on the **validation** and **testing** dataloaders.

In [21]:
# number of images in validation set
len(dataloaders['validation'].sampler)

522

In [22]:
# number of images in test set
len(dataloaders['test'].sampler)

999

You might think that we have a lot of images in test set, but this is because test set consists of repeated images.

Some additional trial information can be observed by accessing the underlying PyTorch dataset object and looking at the `trial_info`. Note that this is not part of the standard PyTorch dataset/dataloader interface, but rather a feature specifically provided by the library!

In [23]:
# Access to the dataset object that underlies all dataloaders
testset = dataloaders['test'].dataset

In [24]:
# get trials that are marked as "test" set
test_trials = np.where(testset.trial_info['tiers'] == 'test')[0]

In [25]:
# get information about the presented image for ALL trials
image_ids = testset.trial_info['frame_image_id']

In [26]:
unique_test_images = np.unique(image_ids[test_trials])
unique_test_images

array([ 104,  128,  183,  355,  479,  483,  656,  803,  830,  936, 1201,
       1494, 1596, 1652, 1656, 1682, 1731, 1756, 1796, 2005, 2008, 2014,
       2159, 2214, 2389, 2586, 2710, 2746, 2747, 2803, 2816, 2825, 2954,
       3018, 3107, 3144, 3163, 3372, 3427, 3438, 3487, 3507, 3562, 3702,
       3847, 3924, 4231, 4295, 4373, 4397, 4400, 4430, 4594, 4619, 4667,
       4674, 4717, 4739, 4782, 4812, 4814, 4821, 4923, 4953, 5034, 5128,
       5166, 5225, 5264, 5288, 5322, 5334, 5399, 5402, 5504, 5640, 5671,
       5679, 5754, 5782, 6013, 6034, 6066, 6082, 6205, 6238, 6248, 6490,
       6562, 6773, 6790, 6831, 6886, 7017, 7028, 7107, 7119, 7120, 7154,
       7495])

In [27]:
len(unique_test_images)

100

We can plot respones of a single neuron to the repeated images

In [28]:
unique_test_images

array([ 104,  128,  183,  355,  479,  483,  656,  803,  830,  936, 1201,
       1494, 1596, 1652, 1656, 1682, 1731, 1756, 1796, 2005, 2008, 2014,
       2159, 2214, 2389, 2586, 2710, 2746, 2747, 2803, 2816, 2825, 2954,
       3018, 3107, 3144, 3163, 3372, 3427, 3438, 3487, 3507, 3562, 3702,
       3847, 3924, 4231, 4295, 4373, 4397, 4400, 4430, 4594, 4619, 4667,
       4674, 4717, 4739, 4782, 4812, 4814, 4821, 4923, 4953, 5034, 5128,
       5166, 5225, 5264, 5288, 5322, 5334, 5399, 5402, 5504, 5640, 5671,
       5679, 5754, 5782, 6013, 6034, 6066, 6082, 6205, 6238, 6248, 6490,
       6562, 6773, 6790, 6831, 6886, 7017, 7028, 7107, 7119, 7120, 7154,
       7495])

In [29]:
np.where(image_ids[test_trials])

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [30]:
trial_idx = test_trials[np.where(image_ids[test_trials] == unique_test_images[0])[0]]

responses = {test_id: [testset[trial][1] for trial in test_trials[np.where(image_ids[test_trials] == test_id)[0]]] for test_id in list(unique_test_images)}


In [31]:
responses

{104: [tensor([1.1952e+00, 1.8324e-09, 1.7112e-01,  ..., 2.3084e-08, 1.1976e-07,
          3.0591e-09], device='cuda:0'),
  tensor([7.7265e-01, 5.9101e-01, 3.8068e-01,  ..., 2.7286e-08, 9.1055e-01,
          3.0758e-09], device='cuda:0'),
  tensor([6.3806e-08, 3.9591e-09, 7.1786e-01,  ..., 1.0790e-08, 2.1358e-08,
          2.3129e-02], device='cuda:0'),
  tensor([1.3310e-01, 1.8952e-01, 4.8420e-01,  ..., 2.4984e-09, 2.3474e-08,
          1.8178e-03], device='cuda:0'),
  tensor([8.2332e-08, 7.0821e-02, 3.0747e-07,  ..., 1.0264e-08, 1.1138e-07,
          2.5770e-01], device='cuda:0'),
  tensor([9.6079e-01, 4.1082e-01, 9.8551e-01,  ..., 4.9020e-03, 3.3084e-03,
          3.0172e-09], device='cuda:0'),
  tensor([4.8258e-08, 8.6106e-10, 3.2044e-08,  ..., 1.8089e-08, 1.0933e-01,
          3.3908e-10], device='cuda:0'),
  tensor([7.9347e-09, 5.0526e-10, 2.4793e-01,  ..., 2.4357e-09, 3.8487e-08,
          3.0661e-10], device='cuda:0'),
  tensor([6.6977e-02, 3.9359e-02, 1.1034e-07,  ..., 1.1486e

In [33]:
trial_idx

array([3095, 3096, 3097, 3098, 3099, 3100, 3101, 3102, 3103, 3104])

So you can see that the test set consists of 100 unique images, each repeated up to 10 times.

In [35]:
testset.trial_info.keys()

['trial_idx',
 'session',
 'frame_trial_ts',
 'frame_last_flip',
 'frame_image_id',
 'frame_image_class',
 'frame_pre_blank_period',
 'condition_hash',
 'tiers',
 'animal_id',
 'scan_idx',
 'frame_presentation_time']

In [36]:
testset.trial_info.frame_image_id  # gives information about presented image ID

array([1301, 5927, 3982, ...,  464,  819, 3025])

In [37]:
testset

FileTreeDataset /content/data/static20457-5-9-preproc0 (n=5993 items)
	images, responses

In [38]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7fb9d6889eb0>,
 'validation': <torch.utils.data.dataloader.DataLoader at 0x7fb9d6889c10>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7fb9d6889ee0>}

In [39]:
len(dataloaders['validation'].dataset.trial_info.frame_image_id)  # gives information about presented image ID

5993

# Modeling the neuronal responses

Now that we have successfully loaded the dataset and inspected its contents, it's time for us to start **modeling** the responses.

We will start by building a very basic **Linear-Nonlinear model** - which is nothing more than a shallow neural network with single linear layer followed by an activation function.

## Linear-Nonlinear (LN) model

### Background

Arguably one of the simplest model of a neuron's response to a stimulus is the **linear-nonlinear (LN) model**. 

Given an image $I \in \mathbb{R}^{h\,\times\,w}$ where $h$ and $w$ are the height and the width of the image, respectively, let us collapse the image into a vector $x \in \mathbb{R}^{hw}$.

A single neuron's response $r$ under linear-nonlinear model can then be expressed as:
$$
r = a(w^\top x + b),
$$
where $w \in \mathbb{R}^{hw}$ and $b \in \mathbb{R}$ are **weight** and **bias**, and $a:\mathbb{R}\mapsto\mathbb{R}$ is a scalar **activating function**.

We can in fact extend to capture the responses of all $N$ neurons simultaneously as:

$$
\mathbf{r} = a(\mathbf{W} x + \mathbf{b}),
$$
where $\mathbf{r} \in \mathbb{R}^{N}$, $\mathbf{W} \in \mathbb{R}^{N\,\times\,hw}$ and $\mathbf{b} \in \mathbb{R}^{N}$.

Hence, each neuron weights each pixel of the image according to the weight $w$ (a column of $\mathbf{W}$) and thus characterizes how much the each neuron "cares" about a specific pixel.

The nonlinear activation function $a(\cdot)$ ensures, among other things, that the output of the network stays above 0. In fitting neuronal responses, we tend to use $a(x) = ELU(x) + 1$ where ELU (Exponential Linear Unit) is defined as follows:

$$
    ELU(x) = 
\begin{cases}
    e^x - 1, & x \lt 0 \\
    x,   & x \ge 0
\end{cases}
$$

In [40]:
# Plotting ELU function
x = np.linspace(-2, 2)
plt.plot(x, F.elu(torch.Tensor(x)))
plt.axhline(0, c='r', ls='--')

We shift it by 1 to ensure it will always remain positive

In [41]:
# Plotting ELU+1 function
x = np.linspace(-2, 2)
plt.plot(x, F.elu(torch.Tensor(x))+1)
plt.axhline(0, c='r', ls='--')

Overall, it can be seen that a linear-nonlinear is nothing more than a single linear layer on flattened image, followed by a nonlinear activation. Now let's go ahead and implment our LN model in PyTorch!

### Implementation

We therefore go ahead and implement a simple network consisting of a linear layer followed by ELU + 1 activation

In [42]:
class Linear(nn.Module):
    def __init__(
        self,
        input_height,
        input_width,
        n_neurons,
        momentum=0.1,
        init_std=1e-3,
        gamma=0.0,
    ):
        super().__init__()
        self.bn = nn.BatchNorm2d(1, momentum=momentum, affine=False)
        self.linear = nn.Linear(input_height * input_width, n_neurons)
        self.gamma = gamma
        self.init_std = init_std
        self.initialize()
        

    def initialize(self, std=None):
        if std is None:
            std = self.init_std
        nn.init.normal_(self.linear.weight.data, std=std)

    def forward(self, x):
        x = self.bn(x)
        x = self.linear(x.flatten(1))
        return nn.functional.elu(x) + 1

    def regularizer(self):
        return self.gamma * self.linear.weight.abs().sum()


And that's it! We have now designed our first network model of the neuron's responses!

**BONUS**: notice that we used batch normalization (BN) layer right before the linear layer? This empirically helps to stabilize the training, allowing us to be not too sensitive to the weight and bias initialization. You could totally implement and train a LN network without such BN layer and you are more than welcome to try! However if you do, be very aware of the network weight initializations and the chocie of learning rate during the training.

Finally, let's instantiate the model before we move onto the next step of training the model!

In [43]:
ln_model = Linear(input_height=64, input_width=36, n_neurons=5335, gamma=0.1)

### Training the network

Now that we have a candidate model designed, it's time to train it. While we could use standard set of optimizers as provided by PyTorch to implement our training routine, here we are provided with a convenience function `train_model` that would handle a lot of the training boiler plate.

In [44]:
from lviv.trainers import train_model

Critically, `train_model` sets up training based on **Poisson loss** and also perform early stopping based on **correlation** of the predicted neuronal responses with the actual neuronal responses on the **validation set**. Let's now talk briefly about our objective (loss) function of choice in training neuron response models - the Poisson loss.

### Mathematical aside: Poisson Loss

#### How we are **actually** modeling the noisy neuronal responses

The use of **Poisson loss** follows from the assumption that, *conditioned on the stimulus*, the neurons' responses follow an **independent Poisson** distribution. That is, given an input image $x$, the population response $\mathbf{r}$ is distributed as:

$$
p(\mathbf{r} | x) = \prod_i^N \text{Poiss}(r_i; \lambda_i(x))
$$

where $r_i$ is the $i^\text{th}$ neuron in the population $\mathbf{r}$. The $\lambda_i$ is the parameter for Poisson distribution that controls its **average value**. Here we express $\lambda_i(x)$ to indicate the fact that the average response for each neuron is expected to vary *as a function of the input image*. We can express this average matching as:

$$
\mathbb{E}[r_i|x] = \lambda_i(x)
$$

In fact, it is precise this function $\lambda_i(x)$ that we are modeling using LN models and, in the next step, more complex neural networks. In otherwords, we are learning $\lambda_i(x) = f_i(x, \theta)$, where $\theta$ is the trainable parameters of the model.

Putting all together, this means that, our model $f(x, \theta)$ is really modeling the average activity of the neurons,

$$
\mathbb{E}[\mathbf{r}|x] = \mathbf{f}(x, \theta)
$$

while we are assuming that the neurons are distribution according to **independent Poisson** distribution around the average responses by our model $\mathbf{f}(x, \theta)$.

#### Deriving the objective function

Poisson distribution is defined as follows:

$$
p(r) = \text{Poiss}(r; \lambda) = \frac{e^{-\lambda}\lambda^{r}}{r!}
$$

During the training, we would want to adjust the model parameter $\theta$ to maximize the chance of observing the response $\mathbf{r}$ to a known image $x$. This is achieved by **maximizing** the log-likelihood function $\log p(\mathbf{r}|x, \theta)$, or equivalently by **minimzing the negative log-likelihood function** as the objective function $L(x, \mathbf{r}, \theta)$:

$$
\begin{align}
L(x, \mathbf{r}, \theta) &= -\log p(\mathbf{r}|x, \theta) \\
&= -\log \prod_i \text{Poiss}(r_i; f_i(x, \theta)) \\
&= -\sum_i \log \frac{e^{-f_i(x, \theta)}f_i(x, \theta)^{r_i}}{r_i!} \\
&= \sum_i \left(f_i(x, \theta) - r_i \log f_i(x, \theta) + \log r_i! \right)
\end{align}
$$


During the optimization, we seek for $\theta$ that would minimize the loss $L$. Note that since the term $log r_i!$ does not depend on $\theta$, it can be safely dropped from Poisson loss. Hence you would commonly see the following expression as the definition of the **Poisson loss**

$$
L_\text{Poiss}(x, \mathbf{r}, \theta) = \sum_i \left(f_i(x, \theta) - r_i \log f_i(x, \theta)\right)
$$

### Performing the training

Now that we have the theoretical foundation for the training and the choice of the objective function under our belt, let's go ahead and train the network. Because the function `train_model` handles a lot underneath the hood, training a model is just as easy as invoking the function by passing it the model to be trained and the dataloaders!

In [45]:
from lviv.trainers import train_model

In [46]:
score, output, model_state = train_model(model=ln_model, dataloader=dataloaders)

correlation -0.0010271524
poisson_loss 9420454.0


Epoch 1: 100%|██████████| 75/75 [00:03<00:00, 23.57it/s]


[001|00/05] ---> 0.06988988816738129
correlation 0.06988989
poisson_loss 3192737.8


Epoch 2: 100%|██████████| 75/75 [00:00<00:00, 125.49it/s]


[002|00/05] ---> 0.07509687542915344
correlation 0.075096875
poisson_loss 2897325.5


...

[028|03/05] -/-> 0.1126239001750946
correlation 0.1126239
poisson_loss 2137293.2


Epoch 29: 100%|██████████| 75/75 [00:00<00:00, 127.29it/s]


[029|04/05] -/-> 0.11358523368835449
correlation 0.11358523
poisson_loss 2134456.0


Epoch 30: 100%|██████████| 75/75 [00:00<00:00, 127.15it/s]


[030|05/05] -/-> 0.1125689372420311
Restoring best model after lr decay! 0.112569 ---> 0.115602
Restoring best model! 0.115602 ---> 0.115602


### Analyzing the trained network

Woohoo! We have now successfully trained our very first LN model on real neuronal responses! But really, how good is the model?

During the training, the `train_model` function iteratively reported two values: the loss function (Poisson loss) value and the average correlation. 

But what is this correlation? It's simply the correlation computed between our predicted neuronal responses $\hat{r}_i$ and the actual neuronal responses $r_i$ across images in the validation set. We then take the average correlation value **across neurons** to get average correlation.

Being a correlation, the highest possible value is of course 1.0. Practically speaking, this is never reached both due to 1) imperfection of our model and more fundamentally due to 2) the noiseness of the neuron's responses. Because of the noise, even a perfect model would never reach a correlation of 1.0.

<font color='red'>
    NOTE TO SELF: Add more here probably plotting some scatter plot for an example neuron, histogram of correlation scores both done on the testset.
</font>

In [47]:
from lviv.utility import measures

# compute correlation on test set
ln_test_correlations = measures.get_correlations(
    ln_model, dataloaders["test"], device="cuda", per_neuron=True
)
n, bins, _ = plt.hist(ln_test_correlations, bins=100, color='k')
plt.vlines(np.median(ln_test_correlations), 0, np.max(n), color="r")
plt.text(
    x=np.median(ln_test_correlations),
    y=np.max(n),
    s="median = {:.3f}".format(np.median(ln_test_correlations)),
)
plt.title('Test correlation of LN model')
plt.xlabel('Correlation')


Text(0.5, 0, 'Correlation')

In [48]:
# noisiness of neural responses: plot one neuron's responses to 20 images (x axis) each shown 10 repeats (y axis)
import pandas as pd
condition_hash = np.load(
    "/content/data/static20457-5-9-preproc0/meta/trials/condition_hash.npy"
)
responses = [
    np.load(f"/content/data/static20457-5-9-preproc0/data/responses/{i}.npy")
    for i in range(len(condition_hash))
]

df = pd.DataFrame({"condition_hash": condition_hash, "response": responses})
df['repeats'] = df.groupby("condition_hash").transform('count')
df = df.loc[df['repeats'] == 10]  # only keep trials with 10 repeats
neuron_idx = 0  # choose one neuron to inspect its responses
for i, (_, repeats) in enumerate(list(df.groupby("condition_hash"))[:20]):
    plt.scatter(np.ones(10) * i, np.stack(repeats.response)[:, neuron_idx], c='k')
plt.xlabel('Images')
plt.ylabel('Responses')

Text(0, 0.5, 'Responses')

## Going beyond Linear-Nonlinear model by using CNN

We saw that a simple LN model can be trained to achieve above chance performance in predicting the responses of mouse V1 neurons to natural images. But we certainly must be able to do better than that, right?

In the past decase, what has really driven system identification in visual neurons has been the use of convolutional neural networks (CNN). Below, we will try out a very simple CNN to see if we can already reach better performance than LN.

<font color='green'>
    NOTE to collaborators: 
    Please add a simpler implementation of CNN. Ideally it would train just as fast as the simple fully connected linear model given above. 
</font>

In [49]:
from collections import OrderedDict
class CNN(nn.Module):
    def __init__(
        self,
        input_height,
        input_width,
        n_neurons,
        momentum=0.1,
        init_std=1e-3,
        gamma=0.1,
        hidden_channels=8,
    ):
        super(CNN, self).__init__()
        self.init_std = init_std
        self.gamma = gamma

        # CNN core
        self.cnn_core = nn.Sequential(
            OrderedDict(
                [
                    ("conv1", nn.Conv2d(1, hidden_channels, 15, padding=15 // 2, bias=False)),
                    ("bn1", nn.BatchNorm2d(hidden_channels, momentum=momentum)),
                    ("elu1", nn.ELU()),
                    ("conv2", nn.Conv2d(hidden_channels, hidden_channels, 13, padding=13 // 2, bias=False)),
                    ("bn2", nn.BatchNorm2d(hidden_channels, momentum=momentum)),
                    ("elu2", nn.ELU()),
                    ("conv3", nn.Conv2d(hidden_channels, hidden_channels, 13, padding=13 // 2, bias=False)),
                    ("bn3", nn.BatchNorm2d(hidden_channels, momentum=momentum)),
                    ("elu3", nn.ELU()),
                    ("conv4", nn.Conv2d(hidden_channels, hidden_channels, 13, padding=13 // 2, bias=False)),
                    ("bn4", nn.BatchNorm2d(hidden_channels, momentum=momentum)),
                    ("elu4", nn.ELU()),
                ]
            )
        )

        # Fully connected readout
        self.readout = nn.Sequential(
            OrderedDict(
                [
                    ('fc_ro', nn.Linear(input_height * input_width * hidden_channels, n_neurons)),
                    ('bn_ro', nn.BatchNorm1d(n_neurons, momentum=momentum)),
                ]
            )
        )


    def initialize(self, std=None):
        if std is None:
            std = self.init_std
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.normal_(m.weight.data, std=std)

    def forward(self, x):
        x = self.cnn_core(x)
        x = x.view(x.size(0), -1)
        x = self.readout(x)
        return nn.functional.elu(x) + 1
    
    def regularizer(self):
        return self.readout[0].weight.abs().sum() * self.gamma


In [50]:
cnn_model = CNN(input_height=64, input_width=36, n_neurons=5335)
score, output, model_state = train_model(model=cnn_model, dataloader=dataloaders)

correlation -0.0001351456
poisson_loss 3343541.0


Epoch 1: 100%|██████████| 75/75 [00:03<00:00, 22.15it/s]


[001|00/05] ---> 0.10400564223527908
correlation 0.10400564
poisson_loss 2577585.2


Epoch 2: 100%|██████████| 75/75 [00:03<00:00, 22.15it/s]


[002|00/05] ---> 0.1374063491821289
correlation 0.13740635
poisson_loss 2076501.0


...

[022|03/05] -/-> 0.19146785140037537
correlation 0.19146785
poisson_loss 1841577.6


Epoch 23: 100%|██████████| 75/75 [00:03<00:00, 22.02it/s]


[023|04/05] -/-> 0.19038550555706024
correlation 0.1903855
poisson_loss 1847065.9


Epoch 24: 100%|██████████| 75/75 [00:03<00:00, 22.04it/s]


[024|05/05] -/-> 0.18596479296684265
Restoring best model after lr decay! 0.185965 ---> 0.207819
Restoring best model! 0.207819 ---> 0.207819


In [51]:
# compute correlation on test set
cnn_test_correlations = measures.get_correlations(
    cnn_model, dataloaders["test"], device="cuda", per_neuron=True
)
n, bins, _ = plt.hist(cnn_test_correlations, bins=100, color='k')
plt.vlines(np.median(cnn_test_correlations), 0, np.max(n), color="r")
plt.text(
    x=np.median(cnn_test_correlations),
    y=np.max(n),
    s="median = {:.3f}".format(np.median(cnn_test_correlations)),
)
plt.title('Test correlation of CNN model')
plt.xlabel('Correlation')

# compare model performance
plt.figure()
plt.scatter(ln_test_correlations, cnn_test_correlations, color="k", s=1)
plt.xlabel('LN correlation')
plt.ylabel('CNN correlation')
plt.title('LN and CNN model performance comparison')
plt.plot([0, 1], [0, 1], color='r')
plt.axis('square')

(-0.14087733291089535,
 1.054327492043376,
 -0.12940694876015185,
 1.0657978761941196)

## Trying out the State-of-the-Art (SOTA) model

Now that we got some sense on how we could go about training linear and nonlinear network models to predict V1 neuron responses to natural images, and we just saw how the nonlinear network seems to bring significant improvement to the performance beyond the LN network.

You might now be wondering, how good can we get? To get a sense of this, let's go ahead and train a state-of-the-art (SOTA) network model for mouse V1 responses to natual images as published in our recent work in [Lurz et al. ICLR 2021](https://github.com/sinzlab/Lurz_2020_code).

To keep things simple, I have provided for the network implementation in the `lviv` package, so you can build the model just by invoking a function!

In [52]:
from lviv.models import build_lurz2020_model
model_config = {'init_mu_range': 0.55,
                'init_sigma': 0.4,
                'input_kern': 15,
                'hidden_kern': 13,
                'gamma_input': 1.0,
                'grid_mean_predictor': {'type': 'cortex',
                                        'input_dimensions': 2,
                                        'hidden_layers': 0,
                                        'hidden_features': 0,
                                        'final_tanh': False},
                'gamma_readout': 2.439
               }

sota_model = build_lurz2020_model(**model_config, dataloaders=dataloaders, seed=1234)

In [53]:
score, output, model_state = train_model(model=sota_model, dataloader=dataloaders)

correlation 0.00034036028
poisson_loss 3467926.2


Epoch 1: 100%|██████████| 75/75 [00:03<00:00, 24.60it/s]


[001|00/05] ---> 0.06803105026483536
correlation 0.06803105
poisson_loss 1933870.8


Epoch 2: 100%|██████████| 75/75 [00:03<00:00, 24.61it/s]


[002|00/05] ---> 0.0992850512266159
correlation 0.09928505
poisson_loss 1907501.2


...

[049|03/05] -/-> 0.29546305537223816
correlation 0.29546306
poisson_loss 1680464.8


Epoch 50: 100%|██████████| 75/75 [00:03<00:00, 24.48it/s]


[050|04/05] -/-> 0.2931968569755554
correlation 0.29319686
poisson_loss 1687673.2


Epoch 51: 100%|██████████| 75/75 [00:03<00:00, 24.35it/s]


[051|05/05] -/-> 0.29394257068634033
Restoring best model after lr decay! 0.293943 ---> 0.296826
Restoring best model! 0.296826 ---> 0.296826


It turns out that we can have *linearized* version of the SOTA model. This effectively removes all nonlinear operations within the network except for the very last nonlinear activation, rendering the network into a **LN model** but with more complex architecture.

In [54]:
linear_model_config = dict(model_config) # copy the config
linear_model_config['linear'] = True # set linear to True to make it a LN model!

sota_ln_model = build_lurz2020_model(**linear_model_config, dataloaders=dataloaders, seed=1234)

In [55]:
score, output, model_state = train_model(model=sota_ln_model, dataloader=dataloaders)

correlation 0.0003725943
poisson_loss 3467901.0


Epoch 1: 100%|██████████| 75/75 [00:03<00:00, 24.95it/s]


[001|00/05] ---> 0.03260115161538124
correlation 0.03260115
poisson_loss 1995799.6


Epoch 2: 100%|██████████| 75/75 [00:02<00:00, 25.17it/s]


[002|00/05] ---> 0.0736672654747963
correlation 0.073667265
poisson_loss 1921495.9


...

[043|03/05] -/-> 0.21505840122699738
correlation 0.2150584
poisson_loss 1781996.5


Epoch 44: 100%|██████████| 75/75 [00:03<00:00, 24.78it/s]


[044|04/05] -/-> 0.21603919565677643
correlation 0.2160392
poisson_loss 1781568.0


Epoch 45: 100%|██████████| 75/75 [00:03<00:00, 24.87it/s]


[045|05/05] -/-> 0.21588590741157532
Restoring best model after lr decay! 0.215886 ---> 0.216275
Restoring best model! 0.216275 ---> 0.216275


### Compare model performance between SOTA and SOTA linear model

In [56]:
# compute correlation on test set
sota_test_correlations = measures.get_correlations(
    sota_model, dataloaders["test"], device="cuda", per_neuron=True
)

sota_ln_test_correlations = measures.get_correlations(
    sota_ln_model, dataloaders["test"], device="cuda", per_neuron=True
)

# compare model performance
plt.figure()
plt.scatter(sota_ln_test_correlations, sota_test_correlations, color="k", s=1)
plt.xlabel('SOTA linear model correlation')
plt.ylabel('SOTA model correlation')
plt.title('SOTA and SOTA linear model performance comparison')
plt.plot([0, 1], [0, 1], color='r')
plt.axis('square')

(-0.12759292609989642,
 1.0536949012428523,
 -0.09512537438422441,
 1.0861624529585243)

# Analyzing the trained model to gain insights into the brain

<font color='green'>
    NOTE to collaborators: 
    Please provide code for generating gradient receptive field and MEI for the sota networks. By this point, they should have `sota_model` and `sota_ln_model` corresponding to the best nonlinear and linear model based on the model architecture as found in Lurz et al. 2021.
</font>

In [76]:
# select neuron to compute RFs and MEIs
neuron_idx = 1
# choose input mean as starting point
input_mean = np.load(
    "/content/data/static20457-5-9-preproc0/meta/statistics/images/all/mean.npy"
).squeeze()
input_std =np.load(
    "/content/data/static20457-5-9-preproc0/meta/statistics/images/all/std.npy"
).squeeze()

In [77]:
def generate_grad_rf(model, starting_point, neuron_idx):
    assert starting_point.ndim == 2
    x = torch.tensor(
        starting_point.copy()[None, None, ...],
        dtype=torch.float,
        device="cuda",
        requires_grad=True,
    )
    model_activation = model(x)[:, neuron_idx]
    model_activation.backward()
    grad_rf = x.grad.data.cpu().numpy().squeeze()
    return grad_rf


# generate a gradient receptive field from the sota_model and the sota_ln_model
grad_rf_sota = generate_grad_rf(
    sota_model, starting_point=np.ones((36, 64)) * input_mean, neuron_idx=neuron_idx
)
grad_rf_sota_ln = generate_grad_rf(
    sota_ln_model, starting_point=np.ones((36, 64)) * input_mean, neuron_idx=neuron_idx
)

# compare the two grad_rfs
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
axes[0].imshow(grad_rf_sota, cmap="gray")
axes[0].set_title("SOTA")
axes[0].axis("off")
axes[1].imshow(grad_rf_sota_ln, cmap="gray")
axes[1].set_title("SOTA_ln")
axes[1].axis("off")
plt.suptitle("Gradient receptive fields", y=0.8)


Text(0.5, 0.8, 'Gradient receptive fields')

In [78]:
from tqdm import tqdm


def generate_mei(
    model,
    starting_point,
    neuron_idx,
    iterations,
    mean_constr,
    std_constr=1,
    lr=1,
    disable_progress_bar=False,
):
    std_constr = torch.tensor(std_constr).cuda()
    mean_constr = torch.tensor(mean_constr).cuda()
    assert starting_point.ndim == 2
    x = torch.tensor(
        starting_point.copy()[None, None, ...],
        dtype=torch.float,
        device="cuda",
        requires_grad=True,
    )
    optimizer = torch.optim.SGD([x], lr=lr, momentum=0.9)
    model_activations = []
    for i in tqdm(range(iterations), disable=disable_progress_bar):
        transformed_x = (x - x.mean()) / (x.std() + 1e-4) * std_constr + mean_constr
        model_activation = model(transformed_x)[:, neuron_idx]
        (-model_activation).backward()
        model_activations.append(model_activation.detach().cpu().numpy())
        optimizer.step()
    x = (x - x.mean()) / (x.std() + 1e-4) * std_constr + mean_constr
    return np.array(model_activations), x.detach().cpu().numpy().squeeze()



In [87]:

# generate a gradient receptive field from the sota_model and the sota_ln_model
activation_sota, mei_sota = generate_mei(
    sota_model,
    starting_point=np.random.normal(input_mean, input_std / 10, (36, 64)),
    neuron_idx=neuron_idx,
    iterations=2000,
    mean_constr=input_mean,
    std_constr=input_std / 10,
    lr=100,
)
activation_sota_ln, mei_sota_ln = generate_mei(
    sota_ln_model,
    starting_point=np.random.normal(input_mean, input_std / 10, (36, 64)),
    neuron_idx=neuron_idx,
    iterations=2000,
    mean_constr=input_mean,
    std_constr=input_std / 10,
    lr=100,
)


100%|██████████| 2000/2000 [00:13<00:00, 151.03it/s]


In [91]:
# compare the two meis
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 5))
axes[0,0].imshow(mei_sota, cmap='gray')
axes[0,0].axis('off')
axes[0,0].set_title('SOTA')
axes[1,0].plot(activation_sota)
axes[1,0].set_ylabel('SOTA model activation')
axes[1,0].set_xlabel('Iterations')
axes[0,1].imshow(mei_sota_ln, cmap='gray')
axes[0,1].axis('off')
axes[0,1].set_title('SOTA_ln')
axes[1,1].plot(activation_sota_ln)
axes[1,1].set_ylabel('SOTA_ln model activation')
axes[1,1].set_xlabel('Iterations')
plt.suptitle('MEIs', y=1.02)

Text(0.5, 1.02, 'MEIs')

### MEI cross activation

In [89]:
neuron_idx_ls = np.random.randint(0, high=5335, size=20)
meis = []
for i in tqdm(neuron_idx_ls):
    _, mei = generate_mei(
            sota_model,
            starting_point=np.random.normal(input_mean, input_std / 10, (36, 64)),
            neuron_idx=i,
            iterations=2000,
            mean_constr=input_mean,
            std_constr=input_std / 10,
            disable_progress_bar=True,
    )
    meis.append(mei)


100%|██████████| 20/20 [04:20<00:00, 13.03s/it]


In [90]:
meis = torch.tensor(np.stack(meis))[:, None, ...].cuda()
activations = sota_model(meis).cpu().detach().numpy().squeeze()[:, neuron_idx_ls]
activations = activations / activations.max(axis=0, keepdims=True)
plt.imshow(activations, cmap='gray')
plt.ylabel('MEIs')
plt.xlabel('Neurons')

Text(0.5, 0, 'Neurons')